<a href="https://colab.research.google.com/github/m-ali25/theme_park/blob/main/Theme_Park_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install python-dotenv

In [2]:
pip install psycopg2

In [4]:
import numpy as np
import requests
import pandas as pd
import psycopg2 as psql
import os
from dotenv import load_dotenv

In [4]:
username = os.getenv('sql_username')
password = os.getenv('sql_password')
my_host = os.getenv('host')

In [8]:
url = 'https://queue-times.com/parks.json'
response = requests.get(url)
parks_data = response.json()

park_id = [x["id"] for park_group in parks_data for x in park_group["parks"]]

In [9]:
def get_ride_details(park_id):
    try:
        url_parks = f'https://queue-times.com/parks.json'
        response_parks = requests.get(url_parks)
        parks_data = response_parks.json()

        url_queue_times = f'https://queue-times.com/parks/{park_id}/queue_times.json'
        response_queue_times = requests.get(url_queue_times)
        queue_data = response_queue_times.json()

        park_name = None
        for park_group in parks_data:
            for park in park_group["parks"]:
                if park['id'] == park_id:
                    park_name = park["name"]
                    break
            if park_name:
                break

        if not park_name:
            return 'Error: Park not found'

        lands = queue_data.get("lands", [])
        ride_details = []

        for land in lands:
            land_name = land["name"]
            rides = land.get("rides", [])

            for ride in rides:
                name = ride.get("name")
                wait_time = ride.get("wait_time")
                ride_status = 'open' if ride.get('is_open') else 'closed'
                ride_details.append((park_name, name, ride_status, wait_time))

        return ride_details

    except:
        return 'Error'

In [10]:
def fetch_parks_data():
    url = 'https://queue-times.com/parks.json'
    response = requests.get(url)
    parks_data = response.json()
    return parks_data

In [11]:
parks_data = fetch_parks_data()

all_ride_details = []

for park_group in parks_data:
    for park in park_group["parks"]:
        park_id = park["id"]
        ride_details = get_ride_details(park_id)
        if ride_details:
            all_ride_details.extend(ride_details)

In [12]:
park_db = pd.DataFrame(all_ride_details,columns = ["park_name", "ride_name", "ride_status", "wait_time"])

In [13]:
park_db.to_csv('park_db.csv')

In [16]:
park_db

,park_name,ride_name,ride_status,wait_time
0,California's Great America,Demon,closed,0
1,California's Great America,Flight Deck,closed,0
2,California's Great America,Gold Striker,closed,0
3,California's Great America,Grizzly,closed,0
4,California's Great America,Patriot,closed,0
...,...,...,...,...
2470,Walt Disney Studios Paris,Slinky® Dog Zigzag Spin,open,10
2471,Walt Disney Studios Paris,Toy Soldiers Parachute Drop,open,50
2472,Walt Disney Studios Paris,Toy Soldiers Parachute Drop Single Rider,open,10
2473,Walt Disney Studios Paris,Armageddon : les Effets Spéciaux,closed,0


In [18]:
conn = psql.connect(database = "pagila",
                    user = username,
                    host = my_host,
                    password = password,
                    port = 5432
                    )

OperationalError: connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?


In [12]:
cur = conn.cursor()

sql = """
INSERT INTO student.park_db (park_name, ride_name, ride_status, wait_time)
VALUES (%s, %s, %s, %s)
"""

cur.executemany(sql, park_db.values)

KeyboardInterrupt: 

In [127]:
conn.commit()
conn.close()

InterfaceError: connection already closed